In [1]:
#import libraries
import os
os.environ["OMP_NUM_THREADS"] = "1"
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,davies_bouldin_score

In [2]:
df=pd.read_csv("spending_l9_dataset.csv")
print(df.head())

   CustomerID  Age  Income_$  SpendingScore  VisitsPerMonth  OnlinePurchases  \
0           1   28        33             78              14                9   
1           2   21        25             87               8               23   
2           3   23        24             88              13               10   
3           4   24        25             73              16               11   
4           5   20        23             88              17               16   

   Gender Region  
0  Female   East  
1    Male  North  
2    Male  South  
3  Female   West  
4    Male   West  


In [3]:
# 2. Select features for clustering
FEATURES = ["Income_$","SpendingScore"]
X = df[FEATURES].copy()

# Fill missing numeric values (if any) with median
for col in FEATURES:
    if X[col].isna().any():
        X[col] = X[col].fillna(X[col].median())
print("Before Scaling the Features")
X.head()


Before Scaling the Features


,Income_$,SpendingScore
0,33,78
1,25,87
2,24,88
3,25,73
4,23,88


In [4]:
# 3. features scalling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# converting the X[Income_$,SpendingScore] Scaled to a DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns = ["Income_$","SpendingScore"])
print("-After Scaling the Features ")
X_scaled_df.head()

-After Scaling the Features 


,Income_$,SpendingScore
0,-0.627830,0.729981
1,-0.890315,1.073591
2,-0.923126,1.111770
3,-0.890315,0.539086
4,-0.955936,1.111770


In [5]:
# 4. K-Means by using (Elbow method)
# this function Calculates and prints the Sum of Squared Errors (SSE) for K-Meansclustering on a given dataset for k values from 1 to 10.
from sklearn.cluster import KMeans

def calculate_sse_for_k(X_scaled):
    print("--- Sum of Squared Errors (SSE) for each value of k ---")
    for k in range(1, 11):  # Isku day k ilaa 10
        print("k =", k)
        km = KMeans(n_clusters=k, n_init="auto", random_state=42)  # K-Means model
        km.fit(X_scaled)  # Tababarka K-Means
        print(f"k={k} → SSE={km.inertia_:.2f}")  # Daabac SSE-ga

# X_scaled waa xogta la habeynayo (scaled_data)
calculate_sse_for_k(X_scaled)

--- Sum of Squared Errors (SSE) for each value of k ---
k = 1
k=1 → SSE=400.00
k = 2
k=2 → SSE=199.70
k = 3
k=3 → SSE=79.37
k = 4
k=4 → SSE=21.37
k = 5
k=5 → SSE=19.09
k = 6
k=6 → SSE=15.65
k = 7
k=7 → SSE=14.48
k = 8
k=8 → SSE=13.81
k = 9
k=9 → SSE=12.94
k = 10
k=10 → SSE=11.52


In [6]:
# 5. Model Training (Pick K)
kmeans = KMeans(n_clusters=4, random_state=42)
# geeting the label
labels = kmeans.fit_predict(X_scaled)
# adding the label to the dataframe 
df["Cluster"] = labels.astype(int)
print("sample with clusters")
df.head()
# print(df["Cluster"].value_counts())


sample with clusters


,CustomerID,Age,Income_$,SpendingScore,VisitsPerMonth,OnlinePurchases,Gender,Region,Cluster
0,1,28,33,78,14,9,Female,East,2
1,2,21,25,87,8,23,Male,North,2
2,3,23,24,88,13,10,Male,South,2
3,4,24,25,73,16,11,Female,West,2
4,5,20,23,88,17,16,Male,West,2


In [7]:
# 6. Evaluating the Clustring
sil = silhouette_score(X_scaled,labels)
dbi = davies_bouldin_score(X_scaled,labels)

print("Metrics: ")
print(f"Silhouette Score: {sil:.2f} (closer to +1 is better)")
print(f"Davies–Bouldin:   {dbi:.2f} (lower is better)")
                           


Metrics: 
Silhouette Score: 0.73 (closer to +1 is better)
Davies–Bouldin:   0.39 (lower is better)


In [8]:
# 7.Cluster Centers (Original Units)
center_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(center_scaled)
print("CLUSTER CENTERS (Original Units) ")
center_df = pd.DataFrame(centers_original,columns=FEATURES)
center_df.head(10)



CLUSTER CENTERS (Original Units) 


,Income_$,SpendingScore
0,56.32,53.58
1,28.92,19.60
2,24.14,83.10
3,99.16,79.24


In [9]:
# 8. Sanity Check
sample_idx = [0,1,2]
sanity = df.loc[sample_idx, FEATURES + ["Cluster"]]
print(" sanity check (3 Customers")
print(sanity)                                        
                                        



 sanity check (3 Customers
   Income_$  SpendingScore  Cluster
0        33             78        2
1        25             87        2
2        24             88        2


In [10]:
# 9. Save Output
df.to_csv("spending_labeled_clusters.csv",index=False)
print("saved ")



SyntaxError: invalid syntax (2323374094.py, line 2)